In [17]:
# Import Libraries
import os
import pinecone
from openai import OpenAI
from dotenv import load_dotenv,find_dotenv
from langchain_openai import OpenAIEmbeddings
from langchain_openai import OpenAI
from langchain_openai import ChatOpenAI
from langchain import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_pinecone import Pinecone

# Load environment variables
load_dotenv(find_dotenv())
client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))
llm = OpenAI(model_name="gpt-3.5-turbo-instruct")

# Import prompt and define PromptTemplate
template = """You are an expert {persona}. Explain the concept of {concept} to a 5 years old kid"""
prompt = PromptTemplate(input_variables=["concept"],template=template,)

print ("Prompt Template:")
print (template)

print ("Prompt:")
print (prompt)

# Run LLM with PromptTemplate
personatext = "doctor"
concepttext = "covid"
explanation = llm.invoke(prompt.format(persona=personatext, concept=concepttext))
print ("LLM Explanation:")
print (explanation)

# Split the output text into smaller chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 100,chunk_overlap  = 0,)
texts = text_splitter.create_documents([explanation])

# Individual text chunks can be accessed with "page_content"
texts[0].page_content

# Import and instantiate OpenAI embeddings
embeddings = OpenAIEmbeddings()

# Turn the first text chunk into a vector with the embedding
index_name = 'search'
search = Pinecone.from_documents(index_name = index_name, documents = texts, embedding = embeddings)

# Do a simple vector similarity search
query = "What is the best medicine for covid?"
result = search.similarity_search(query)
print(result)


Prompt Template:
You are an expert {persona}. Explain the concept of {concept} to a 5 years old kid
Prompt:
input_variables=['concept', 'persona'] template='You are an expert {persona}. Explain the concept of {concept} to a 5 years old kid'
LLM Explanation:


Hello there, little one! My name is Dr. Smith and I'm an expert doctor. I'm here to explain to you what Covid is in a way that you can understand.

Covid is a big sickness that can make people feel very sick. It's caused by a tiny germ called a virus that can make us sick. Just like how you sometimes catch a cold or a flu, people can catch Covid too.

But don't worry, there are things we can do to keep from getting sick. We can wash our hands often and wear a mask to cover our nose and mouth. This helps to keep the germs away. We can also stay away from people who are sick to keep ourselves healthy.

Sometimes, people who catch Covid have to go to the hospital to get better. But most people just have to stay at home and rest until